# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for GPU

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Get helper functions


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-10-10 23:14:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-10 23:14:31 (61.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle Introduction to NLP dataset (text samples of Tweets labelled as dister or not disaster).

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-10-10 23:14:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.48, 172.217.1.208, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.04s   

2021-10-10 23:14:34 (13.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do would be to use Python. 

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle trianing dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [8]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher that total num samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
    print(f'Text:\n{text}\n')
    print('---\n')

Target: 0 (not real disaster)
Text:
Snowstorm planned outside #Rome's St Mary Major tonight - annual occasion artificial snow remembering summer snow in 358 AD on same spot.

---

Target: 0 (not real disaster)
Text:
you could slit my throat and I'd apologize for bleeding on you

---

Target: 0 (not real disaster)
Text:
@RebeccaforReal accepts Wisconsin Emergency Response Plan on behalf of @GovWalker #nbc15 http://t.co/Pis0aiVRbR

---

Target: 1 (real disaster)
Text:
RT @LivingSafely: NWS posts Severe #Thunderstorm Warnings for parts of #AR #NC #OK. Seek strong shelter if at risk: http://t.co/kEa5l3b1AE

---

Target: 0 (not real disaster)
Text:
You will be held hostage by a radical group.

---



## Split data into trianing and validation sets

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# Use train_test_split to split trianing data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'],
                                                                            train_df_shuffled['target'],
                                                                            test_size = 0.1, # use 10% data for val split
                                                                            random_state = 42) 

In [11]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(5921    @mogacola @zamtriossu i screamed after hitting...
 3904              Imagine getting flattened by Kurt Zouma
 2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
 3718    @shakjn @C7 @Magnums im shaking in fear he's g...
 1667    Somehow find you and I collide http://t.co/Ee8...
 4435    @EvaHanderek @MarleyKnysh great times until th...
 2544                     destroy the free fandom honestly
 7223    Weapons stolen from National Guard Armory in N...
 4265    @wfaaweather Pete when will the heat wave pass...
 6568    Patient-reported outcomes in long-term survivo...
 Name: text, dtype: object, 5921    0
 3904    0
 2804    1
 3718    0
 1667    0
 4435    1
 2544    1
 7223    0
 4265    1
 6568    1
 Name: target, dtype: int64)

## Converting text to numbers

When dealing with text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character to number)
* Embedding - Create a matrix of feature vector to each token (the size of the feature vector can be defined adn this embedding can be learned). 

### Text vectorization (tokenization)

In [13]:
train_sentences[:5]

5921    @mogacola @zamtriossu i screamed after hitting...
3904              Imagine getting flattened by Kurt Zouma
2804    @Gurmeetramrahim #MSGDoing111WelfareWorks Gree...
3718    @shakjn @C7 @Magnums im shaking in fear he's g...
1667    Somehow find you and I collide http://t.co/Ee8...
Name: text, dtype: object

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # Create group of n-words
                                    output_mode='int', # how to map token to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    )

In [15]:
# Find the average number of tokens (words) int the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to ahve in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a ranodm sentence from training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence} \
      \n \nVectorized version:')
text_vectorizer([random_sentence])

Original text: 
 I will never support looting or the burning of buildings but when seeing the people fight back against the police. I was proud       
 
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   8,   38,  192,  724, 2403,   53,    2,   86,    6,   95,   30,
          45,  989,    2,   57]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words  = words_in_vocab[-5:] # least common words
print(f'Number of words in vocab:{len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab:10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating and Embedding using an Embedding Layer

To make our emebedding, we're going to user TF Embedding Layer.

The parameters we care most abour for our embedding layer:
* `input_dim` = The size of our vocabulary
* `output_dim` = The size of the output embedding vector, for example a value of a 100 would mean each token gets represented by a vector of 100 long.
* `input_length` = length of sequences being passed to the embedding layer.



In [21]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # output shape
                             embeddings_initializer = 'uniform', 
                             input_length = max_length, # how long is each input
                             )

embedding 

In [22]:
# Get a random senteces from the training set
random_sentence = random.choice(train_sentences)
print(f'Original text: \n {random_sentence} \
        \n \n Embedded verison:')

# Embed the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
 Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/81HVV3N3rS         
 
 Embedded verison:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-4.2723764e-02, -5.4419041e-05, -1.1071742e-02, ...,
         -2.7405942e-02, -5.4445639e-03, -3.2623373e-02],
        [ 6.4245090e-03,  4.2477611e-02, -3.2706037e-03, ...,
         -3.1105591e-02,  4.0494550e-02,  4.2808224e-02],
        [ 2.4101999e-02, -2.8186012e-02,  1.8832687e-02, ...,
         -5.4300539e-03,  3.6775734e-02, -1.6044367e-02],
        ...,
        [-2.8103029e-02,  4.6884511e-02, -4.2112209e-02, ...,
         -3.6690664e-02,  3.2403860e-02, -8.3427653e-03],
        [ 1.4040265e-02,  2.5271144e-02, -4.3340363e-02, ...,
         -1.3233554e-02, -2.3234356e-02, -3.3080459e-02],
        [ 2.8412532e-02,  7.2720274e-03, -2.3067482e-03, ...,
         -4.0883645e-03,  2.9895578e-02,  1.8992536e-03]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-4.2723764e-02, -5.4419041e-05, -1.1071742e-02,  4.0284980e-02,
        -3.0447423e-02, -2.4434507e-02,  2.0805087e-02,  3.2905173e-02,
         3.4063365e-02,  4.4952940e-02,  7.7425353e-03,  4.4467915e-02,
        -2.0347321e-02,  3.5413492e-02, -4.3269038e-02,  3.9714646e-02,
         2.6216913e-02, -3.1502724e-02,  2.8959516e-02, -4.3084752e-02,
        -2.3546070e-04, -4.7074329e-02, -2.7078832e-02, -2.5795901e-02,
        -4.6517622e-02, -4.6599377e-02,  3.7145209e-02, -1.0971509e-02,
        -3.7741542e-02, -1.3732184e-02, -2.7583266e-02, -3.8279545e-02,
        -1.6261339e-02, -3.3228219e-02, -3.5865832e-02,  4.9698997e-02,
        -1.6177129e-02,  3.9385010e-02, -2.2928441e-02,  3.1048406e-02,
         1.8178586e-02,  3.9596107e-02,  4.1701172e-02, -2.2233760e-02,
         3.2724962e-03,  3.6143791e-02, -1.3003923e-02,  2.1484978e-03,
         3.4500387e-02, -3.3748068e-02,  4.4992566e-03,  2.1978904e-02,
        -6.8293

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D convolutional neural netwrok (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data.

How are we going to approach all of these?
Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model 
* Evaluate a model


### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a becnhmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the Tf-idf formula to convert our words to numbers

> 🔑 ** Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and later using DL if you can improve upon them.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [25]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%')

Our baseline model achieves an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
train_df.target.value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

### Creating an evaluation function for our model experiments

We could evaluate all of our model's prediction with different metrics every time, however, this would be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with the truth label with the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score

In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    '''
    Calculates model accuracy, precision, recall and F1-score of binary classification model.
    '''
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred)*100
    # Calculate model precision, recall and F1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
    model_results = {'accuracy':model_accuracy,
                     'precision':model_precision*100,
                     'recall':model_recall*100,
                     'model_f1':model_f1*100}
    return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'model_f1': 78.6218975804955,
 'precision': 81.11390004213173,
 'recall': 79.26509186351706}

### Model 1: Feed-forward neural network (dense model)

In [31]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = 'model_logs'

In [33]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape =(1, ), dtype= tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # turn numbers into embedding
# x = layers.GlobalAveragePooling1D(name='global_average_pooling')(x)
x = layers.GlobalMaxPool1D(name='global_max_pooling')(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation='sigmoid')(x) # Create output layer, want binary outputs so use sigmoid
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_max_pooling (GlobalMa (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile model
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [36]:
# Fit the model
model_1_history = model_1.fit(x = train_sentences,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                       experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20211010-231609
Epoch 1/5
215/215 [==============================] - 5s 20ms/step - loss: 0.6356 - accuracy: 0.6490 - val_loss: 0.5708 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4715 - accuracy: 0.8256 - val_loss: 0.4723 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3425 - accuracy: 0.8780 - val_loss: 0.4497 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2581 - accuracy: 0.9101 - val_loss: 0.4526 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1972 - accuracy: 0.9336 - val_loss: 0.4653 - val_accuracy: 0.7913


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4653 - accuracy: 0.7913


[0.46532729268074036, 0.7913385629653931]

In [38]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [39]:
# look at a single prediction
model_1_pred_probs[0]

array([0.40794387], dtype=float32)

In [40]:
# Look at the first 10 preds
model_1_pred_probs[:10]

array([[0.40794387],
       [0.8590145 ],
       [0.99398243],
       [0.06834802],
       [0.19819999],
       [0.9559652 ],
       [0.8955252 ],
       [0.9884811 ],
       [0.9319416 ],
       [0.089647  ]], dtype=float32)

In [41]:
# convert model predictions probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [42]:
# Calculate our model_1 results 
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred= model_1_preds)
model_1_results

{'accuracy': 79.13385826771653,
 'model_f1': 78.83318764387542,
 'precision': 79.6653347825067,
 'recall': 79.13385826771653}

## Visualizing learned embeddings 

In [43]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_max_pooling (GlobalMa (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Get the weight matrix of embedding layer
# (these are the numerical reprsentations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output dim of embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, lets see how we can visualize it.

To do so, Tensorflow has a handy tool called Tensorflow projector: https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeddings: https://www.tensorflow.org/text/guide/word_embeddings

In [46]:
# Create embedding files (we got this from Tensorflow Words embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [47]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural network (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long short term memory (one of the most popular LSTM cells)

Our strucutre of an RNN typically looks like this:
```
Input (text) -> tokenize -> embedding -> layers (RNNs/Dense) -> Output (label probability)


In [51]:
# Create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True, )(x) # when you're stacking RNN cells together, you need to return sequence s
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')


In [52]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,333,633
Trainable params: 1,333,633
Non-trainable params: 0
____________________________________________

In [53]:
# Compile the model
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [54]:
# Fit the model
model_2.fit(x = train_sentences,
            y = train_labels,
            epochs = 5,
            validation_data = (val_sentences, val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                     'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211010-235555
Epoch 1/5
215/215 [==============================] - 9s 32ms/step - loss: 0.3842 - accuracy: 0.8291 - val_loss: 0.4734 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.2513 - accuracy: 0.8983 - val_loss: 0.5312 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.1806 - accuracy: 0.9334 - val_loss: 0.5841 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 6s 29ms/step - loss: 0.1374 - accuracy: 0.9512 - val_loss: 0.6326 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.1082 - accuracy: 0.9631 - val_loss: 0.8022 - val_accuracy: 0.7520


In [55]:
# make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[0.20830902],
       [0.95058036],
       [0.9998549 ],
       [0.08066821],
       [0.04295838],
       [0.9999249 ],
       [0.8514508 ],
       [0.9999801 ],
       [0.9999541 ],
       [0.72175884]], dtype=float32)

In [56]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [57]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy': 75.19685039370079,
 'model_f1': 75.23735887402714,
 'precision': 75.39200264275287,
 'recall': 75.19685039370079}